In [44]:
from pyspark.sql import SparkSession
import pickle
import joblib
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import pandas as pd
import numpy as np
# Use a raw string to handle backslashes in Windows paths
jdbc_jar_path = r"D:\Programming\jdbc6\sqljdbc_6.0\enu\jre8\sqljdbc42.jar"

spark = SparkSession.builder \
    .appName("SQLServerConnection") \
    .config("spark.jars", jdbc_jar_path) \
    .getOrCreate()

# Connection details with trustServerCertificate=true
jdbc_url = "jdbc:sqlserver://DESKTOP-FCEDQJG:1433;databaseName=cancer;trustServerCertificate=true"
table = "patient_data"

# Replace with your SQL Server credentials
df = spark.read.format("jdbc").options(
    url=jdbc_url,
    driver="com.microsoft.sqlserver.jdbc.SQLServerDriver",
    dbtable=table,
    user="alialkady",
    password="Aa22540444"
).load()


In [46]:
# Load the pre-trained model
with open('models/logistic_regression.pkl', 'rb') as file:
    model = joblib.load(file)  # Load the model

# Verify the model type
print(f"Model type: {type(model)}")  # Should be a scikit-learn model

# Load data from SQL Server into Spark DataFrame
df = spark.read.format("jdbc").options(
    url=jdbc_url,
    driver="com.microsoft.sqlserver.jdbc.SQLServerDriver",
    dbtable=table,
    user="alialkady",
    password="Aa22540444"
).load()

# Define your feature columns (same as used during training)
feature_columns = ['coughing_of_blood', 'alcohol_use', 'passive_smoker', 
                   'obesity', 'smoking', 'balanced_diet', 'chest_pain', 
                   'fatigue', 'air_pollution', 'genetic_risk']  # Adjust as necessary

# Extract features and convert to Pandas DataFrame
features = df.select(*feature_columns).toPandas()  # Convert Spark DataFrame to Pandas DataFrame

# Make predictions using the loaded model
predictions = model.predict(features)  # This should work if model is correct

# Check the predictions type and shape
print(f"Predictions type: {type(predictions)}, shape: {predictions.shape}")  # Check predictions

# Add predictions to the original Spark DataFrame
# Convert predictions to a Spark DataFrame
predictions_df = spark.createDataFrame(pd.DataFrame(predictions, columns=["predictions"]))

# Combine original DataFrame with predictions
df_with_predictions = df.crossJoin(predictions_df)

# Show results
predictions_df.show()
spark.stop()

Model type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
Predictions type: <class 'numpy.ndarray'>, shape: (1000,)
+-----------+
|predictions|
+-----------+
|          1|
|          1|
|          2|
|          2|
|          2|
|          2|
|          1|
|          0|
|          1|
|          0|
|          2|
|          2|
|          1|
|          2|
|          0|
|          1|
|          1|
|          2|
|          2|
|          1|
+-----------+
only showing top 20 rows



In [52]:
# Load the pre-trained model
with open('models/svm.pkl', 'rb') as file:
    model = joblib.load(file)  # Load the model


# Load data from SQL Server into Spark DataFrame
df = spark.read.format("jdbc").options(
    url=jdbc_url,
    driver="com.microsoft.sqlserver.jdbc.SQLServerDriver",
    dbtable=table,
    user="alialkady",
    password="Aa22540444"
).load()

# Define your feature columns (same as used during training)
feature_columns = ['coughing_of_blood', 'alcohol_use', 'passive_smoker', 
                   'obesity', 'smoking', 'balanced_diet', 'chest_pain', 
                   'fatigue', 'air_pollution', 'genetic_risk']  # Adjust as necessary

# Extract features and convert to Pandas DataFrame
features = df.select(*feature_columns).toPandas()  # Convert Spark DataFrame to Pandas DataFrame

# Make predictions using the loaded model
predictions = model.predict(features)  # This should work if model is correct

# Check the predictions type and shape
print(f"Predictions type: {type(predictions)}, shape: {predictions.shape}")  # Check predictions

# Add predictions to the original Spark DataFrame
# Convert predictions to a Spark DataFrame
predictions_df = spark.createDataFrame(pd.DataFrame(predictions, columns=["predictions"]))

# Combine original DataFrame with predictions
df_with_predictions = df.crossJoin(predictions_df)

# Show results
predictions_df.show()
spark.stop()

Model type: <class 'sklearn.svm._classes.SVC'>
Predictions type: <class 'numpy.ndarray'>, shape: (1000,)
+-----------+
|predictions|
+-----------+
|          1|
|          1|
|          2|
|          2|
|          2|
|          2|
|          1|
|          0|
|          1|
|          1|
|          2|
|          2|
|          1|
|          2|
|          0|
|          1|
|          1|
|          2|
|          2|
|          1|
+-----------+
only showing top 20 rows

